# Used to calculate the exact tunespread of a shot from measured data 
This script uses the tunespread tool and the Mathematica notebook (used to collect data from the tomo dat files) both written by Adrian Oeftiger (CERN BE-ABP-HSC)

In [13]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from io import StringIO
lorentz_beta = 0.91444281513833
lorentz_gamma = 2.4708737618826    
from scipy.constants import m_p, m_e, speed_of_light, e
r_p = 2.8179403267e-15 *m_e/m_p

# Tomo helper functions

In [14]:
## variables saved in tomoXYZ_eval.dat files in order of line
long_eval_headers = [
    'matchedemittance', 'ninetyemittance', 'rmsemittance', 
    'statisticalemittance', 'bunchingfactor', 'fs0', 'fs1', 
    'eperimage', 'peakcurrent', 'peakdensity', 'sigmaE', 
    'deltap', 'bunchlength', 'dtbin', 'dEbin', 'phasespace',
]

def extract_ctime_harmonic(fname, data):
    with open(fname + '.dat', 'r') as stream:
        stream.readline() # first line
        data['time'] = int(stream.readline()) # second line
        for i in range(67):
            stream.readline()
        data['harmonic'] = int(stream.readline()) # 69th line
        data['lshape'] = data['bunchingfactor'] #/ data['harmonic']
    #print 'harmonic = ', data['harmonic'], ' time of file = ', data['time'], 'lshape = ', data['lshape']
    return data

def extract_long_eval(ftime):
    data = {}
    if '_eval.dat' in ftime:
        pass
    elif '.dat' in ftime:
        ftime = ftime[:-4]
        ftime = ftime + '_eval.dat'
    else:
        ftime = ftime + '_eval.dat'
        
    with open (ftime) as stream :
        for header in long_eval_headers[:-1]:
            data[header] = float(stream.readline())
    extract_ctime_harmonic(ftime[:-9], data)

    return data

def extract_long_phasespace(ftime):
    #with open(ftime + '_tomo{ctime}_eval.dat'.format(ctime=ctime), 'r') as stream:
    with open(ftime + '_eval.dat', 'r') as stream:
        fcontent = stream.readlines()
    phasespace = fcontent[15][2:-2]
    phasespace = (phasespace.replace('}, {', '\n')
                            .replace('*^', 'e'))
    phasespace = StringIO(unicode(phasespace))
    return np.genfromtxt(phasespace, dtype=np.float64, delimiter=',')

def extract_deltap_profile(ftime):
    #profile = np.genfromtxt(ftime + '_tomo{ctime}_deltap.dat'.format(ctime=ctime), delimiter=',', dtype=np.float64, unpack=True)
    profile = np.genfromtxt(ftime + '_deltap.dat', delimiter=',', dtype=np.float64, unpack=True)
    return profile

def extract_fildeltap_profile(ftime):
    #profile = np.genfromtxt(ftime + '_tomo{ctime}_fildeltap.dat'.format(ctime=ctime), delimiter=',', dtype=np.float64, unpack=True'):
    profile = np.genfromtxt(ftime + '_fildeltap.dat', delimiter=',', dtype=np.float64, unpack=True)
    return profile

def extract_intensities(myDataStruct, window_radius_ms=3):
    ctime = 185
    t_offset = myDataStruct.PR_BCT_ST.Samples.value.firstSampleTime
    lo, hi = (ctime - window_radius_ms - t_offset, 
              ctime + window_radius_ms - t_offset)
    intensity_185 = 1e10 * np.mean(myDataStruct.PR_BCT_ST.Samples.value.samples[lo:hi+1])
    ctime = 1350
    t_offset = myDataStruct.PR_BCT_ST.Samples.value.firstSampleTime
    lo, hi = (ctime - window_radius_ms - t_offset, 
              ctime + window_radius_ms - t_offset)
    intensity_1350 = 1e10 * np.mean(myDataStruct.PR_BCT_ST.Samples.value.samples[lo:hi+1])
    return intensity_185, intensity_1350

def extract_long_profile(myDataStruct_scopechannel):
    '''Return time position [in ns] and normalised profile [integrates to 1].'''
    dtbin = myDataStruct_scopechannel.Acquisition.value.sampleInterval 
    tomodata = myDataStruct_scopechannel.Acquisition.value.value
    tomotimes = np.arange(0, len(tomodata[0]) * dtbin, dtbin)
    tomo_reference = tomodata[0]
    baseline = np.mean(tomo_reference[:int(len(tomo_reference)*0.05)])
    tomoprofile = tomo_reference - baseline
    tomoprofile /= np.trapz(tomoprofile, tomotimes)

    return tomotimes, tomoprofile

def plot_longphasespace(ftime, ax=None, *contourf_args, **contourf_kwargs):
    longdata = extract_long_eval(ftime)
    longdata['phasespace'] = extract_long_phasespace(ftime)
    elen, tlen = longdata['phasespace'].shape
    thalf = tlen*longdata['dtbin']/2.
    ehalf = elen*longdata['dEbin']/2.
    TT, EE = np.meshgrid(np.linspace(-thalf, thalf, tlen), 
                         np.linspace(-ehalf, ehalf, elen))
    if ax is None:
        ax = plt.gca()
    ax.contourf(TT, EE, longdata['phasespace'].T, origin='lower', 
                cmap=plt.get_cmap('hot_r'),
                *contourf_args, **contourf_kwargs)
    ax.set_aspect(thalf/ehalf)
    plt.grid(True)

#long_data = extract_ctime_harmonic('/eos/user/h/harafiqu/SWAN_projects/PS/MD4224_2018_10_22/tomo/2018-10-16_012',long_data)
#print long_data
#plot_longphasespace('/eos/user/h/harafiqu/SWAN_projects/PS/MD4224_2018_10_22/tomo/2018-10-16_012')


def plot_c185_c1350_longphasespace(ftime):
    fig, axes = plt.subplots(1, 2, figsize=(11,5), sharex=True, sharey=True)

    plot_longphasespace(ftime[:-4], ctime=185, ax=axes[0])
    axes[0].set_title('C185')
    plot_longphasespace(ftime[:-4], ctime=1350, ax=axes[1])
    axes[1].set_title('C1350')

    for ax in axes:
        ax.grid(True)

def plot_c185_c1350(what, avg=True, fig=None):
    if fig:
        ax = plt.gcf(fig).axes
    else:
        fig, ax = plt.subplots(1, 2, figsize=(12,5), sharex=True, sharey=True)
    plt.sca(ax[0])
    ax[0].set_title('C185')
    ax[0].plot(data_gauss[what][0], label='gauss')
    ax[0].plot(data_hollow[what][0], label='hollow')
    if avg:
        ax[0].axhline(np.mean(data_gauss[what][0]), c='b', ls='--')
        ax[0].axhline(np.mean(data_hollow[what][0]), c='g', ls='--')
    ax[0].grid(True)
    plt.legend()
    
    plt.sca(ax[1])
    ax[1].set_title('C1350')
    ax[1].plot(data_gauss[what][1], label='gauss')
    ax[1].plot(data_hollow[what][1], label='hollow')
    if avg:
        ax[1].axhline(np.mean(data_gauss[what][1]), c='b', ls='--')
        ax[1].axhline(np.mean(data_hollow[what][1]), c='g', ls='--')
    ax[1].grid(True)
    plt.legend()
    plt.suptitle(what, fontsize=18)
    plt.subplots_adjust(top=0.85)

In [15]:
def read_tfs_table_return_data(abs_path, beta=0.91444281513833, gamma=2.4708737618826):

    madx_file_absolute_path = abs_path
    with open(madx_file_absolute_path, 'r') as madx_file:
        # New method - search for 'TIME' in file
        for line in xrange(80): 
            if 'TIME' in madx_file.readline():
                madx_keywords_line = int(line)+1
                break     

        # Save MADX keywords as data labels
        labels = madx_file.readline()[1:-1]
#        print 'read_tfs_table_return_data::', len(labels.split()), ' labels found in MADX TWISS file'

        madx_file.seek(madx_keywords_line+1)    

        # All we need is S, BETX, BETY, DX, DY, and the Circumference
        beta_x_col= -1
        beta_y_col= -1
        d_x_col = -1
        d_y_col = -1
        s_col = -1
        for l in range(len(labels.split())):
            if labels.split()[l] == ('BETX'): beta_x_col = l
            if labels.split()[l] == ('BETY'): beta_y_col = l
            if labels.split()[l] == ('DX'): d_x_col = l
            if labels.split()[l] == ('DY'): d_y_col = l
            if labels.split()[l] == ('S'): s_col = l

    def file_len(fname):
        with open(fname) as f:
            for counter, value in enumerate(f):
                pass
        print '\n', fname, ' has ', counter+1, ' lines'
        return counter + 1

    madx_file_lines = file_len(madx_file_absolute_path)

    # open twiss file again
    with open(madx_file_absolute_path, 'r') as madx_file:
        # read past the header lines
        for x in range(madx_keywords_line+2):        
            madx_file.readline() 

        circumference = 0.0
        remaining_lines = madx_file_lines - (madx_keywords_line + 2)
        s = np.empty(remaining_lines, dtype=float)
        beta_x = np.empty(remaining_lines, dtype=float)
        beta_y = np.empty(remaining_lines, dtype=float)
        d_x = np.empty(remaining_lines, dtype=float)
        d_y = np.empty(remaining_lines, dtype=float)

        # read in data line by line to fill dictionary
        for x in range(remaining_lines):   
            line = madx_file.readline()
            #circumference = circumference + float(line.split()[s_col])    
            s[x] = float(line.split()[s_col])    
            beta_x[x] = float(line.split()[beta_x_col])
            beta_y[x] = float(line.split()[beta_y_col])
            d_x[x] = float(line.split()[d_x_col]) * lorentz_beta
            d_y[x] = float(line.split()[d_y_col]) * lorentz_beta

    # Now we have arrays for betax betay dx dy and a value for the circumference  
    # Let's put them in the expected dicitonary
    myData = {}

    myData['s'] = s
    myData['beta_x'] = beta_x
    myData['beta_y'] = beta_y
    myData['d_x'] = d_x
    myData['d_y'] = d_y

    # Alternatively we can create a multidimensional numpy array
    #twiss_data = np.column_stack((s, beta_x, beta_y, d_x, d_y))

#    print 'read_tfs_table_return_data: complete. Returning data dictionary for s, beta_x, beta_y, d_x, and d_y' 
    return myData

In [16]:
def make_tst_inputs(intensity, deltap, emit_geo_x, emit_geo_y, std_x_div_by_Dx, lorentz_beta=0.91444281513833, lorentz_gamma=2.4708737618826, mass=m_p):
    return dict(
        mass = (mass * speed_of_light**2/e * 1e-9),
        beta = lorentz_beta,
        gamma = lorentz_gamma,
        n_part = int(intensity),
        deltap = deltap,
        emit_geom_x = emit_geo_x,
        emit_geom_y = emit_geo_y,
        n_charges_per_part = 1,
        std_x_div_by_Dx = std_x_div_by_Dx,
    )
#make_tst_inputs = np.vectorize(make_tst_inputs)

In [17]:
from scipy.constants import epsilon_0, c, m_p

In [18]:
e**2/(4*np.pi*epsilon_0*m_p*c**2)

1.5346982646272323e-18

In [19]:
r_p

1.5346982671813621e-18

In [20]:
def calculate_exact_tunespread(data, inputs, tomodata, beta=0.91444281513833, gamma=2.4708737618826, f_verbose=False):
    """Calculates the (maximum) tune shift DeltaQ_x and DeltaQ_y due
    to space charge with given optics parameters provided by
    the dictionaries / hash tables <i>inputs</i> and <i>data</i>.
    Assume coasting so we don't need sig_z
    """
#     std_x_div_by_Dx is the horizontal profile's standard deviation
#     divided by the horizontal dispersion at that point. 
#     Equivalently for the vertical plane with std_y_div_by_Dy.
    r = r_p
    n_part = inputs["n_part"]
    emit_x = inputs["emit_geom_x"]
    emit_y = inputs["emit_geom_y"]
    #lshape = inputs.get("lshape", 1.) #from tomo
    lshape = 1 / tomodata['bunchingfactor']
    std_x_div_by_Dx = inputs["std_x_div_by_Dx"] #from gaussian fit + optics_file
    integx = integy = 0
    
    ds = np.diff(data["s"])
    beta_x = data["beta_x"][:-1] #from optics file
    beta_y = data["beta_y"][:-1] #from optics file
    d_x = data["d_x"][:-1] #from optics file

    # dispersion only scales the convolution (joint PDF of x = x_beta + D_x * dp)
    sqx = std_x_div_by_Dx * d_x
    sqy = np.sqrt(emit_y * beta_y)
    integx = np.sum(beta_x * ds / (sqx * (sqx + sqy)))
    integy = np.sum(beta_y * ds / (sqy * (sqx + sqy)))    
    
    prefactor = r * n_part / (2.0 * np.pi * beta**2 * gamma**3)

    circumference = data["s"][-1]
    shapefactor = lshape / circumference
    #lambda_max_norm = tomodata['peakcurrent'] / (beta * speed_of_light * e) / tomodata['eperimage']
    
    
    print 'integx =', integx, ', integy = ', integy, 'prefactor = ', prefactor, ', shapefactor = ', shapefactor, 'circumference = ', circumference #'lamda_max_norm = ', lambda_max_norm
    print 'sig_x = ', sqx[1], 'ig_y = ', sqy[1]
    print 'beta_x =', beta_x[1], 'beta_y =', beta_y[1], 'd_x =', d_x[1], 'max(ds) = ', max(ds), 'std_x_div_by_Dx = ', std_x_div_by_Dx

    
    DeltaQ_x = prefactor * shapefactor * integx
    DeltaQ_y = prefactor * shapefactor * integy
    
#     DeltaQ_x = circumference * lambda_max_norm * prefactor * shapefactor * integx
#     DeltaQ_y = circumference * lambda_max_norm * prefactor * shapefactor * integy    
    
    return (DeltaQ_x, DeltaQ_y)

calc_tune_spread = np.vectorize(lambda tst_inputs: calculate_exact_tunespread(tst_data, tst_inputs, False))